Rules:

In [61]:
rem_warnings = True

Investment parameters:

In [62]:
horizon = 72
horizon_max = 156 #3Y
scale = 56 #1Y

Statistical parameters:

In [63]:
p = 0.95

Drop warnings:

In [64]:
import warnings
if rem_warnings is True:
    warnings.filterwarnings('ignore')

Libraries:

In [65]:
import pandas as pd
import numpy as np
import scipy as sc
import math
from matplotlib import pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf

Read data:

In [66]:
df = pd.read_csv('data/sap.csv', delimiter=',', names=['date', 'time', 'open', 'high', 'low', 'close', 'volume'])
df.dropna(how='any', axis=0, inplace=True)

Transform data:

In [67]:
df = df[['date', 'close']]

Returns definition:

In [68]:
df['return'] = df['close'].apply(np.log)
df['return'] = 100*df['return'].diff(periods=horizon)
df = df.tail(len(df) - horizon_max)

Dates:

In [69]:
print('Start date:', str(list(df['date'])[0]))
print('End date:', str(list(df['date'])[-1]))

Start date: 1999.05.02
End date: 2019.10.20


Description:

In [70]:
df.describe()

,close,return
count,1059.000000,1059.000000
mean,1556.148376,6.269770
std,564.059811,20.218126
min,683.920000,-78.569272
25%,1150.785000,-0.598265
50%,1354.940000,12.174848
75%,1953.035000,19.890792
max,3025.910000,47.699566


Statistics:

In [71]:
med_return = int(round(df['return'].median()))
var_risk = int(round(df[df['return'] < 0]['return'].abs().quantile(p)))

Show statistics:

In [72]:
print('Med. return:', med_return, '%')
print('Value at Risk:', var_risk, '%')
print('Return at Risk:', round(med_return/var_risk, 2))
print('Autocorrelation:' ,int(round(100*df['return'].autocorr(lag=1))), '%')

Med. return: 12 %
Value at Risk: 55 %
Return at Risk: 0.22
Autocorrelation: 99 %
